# Spring 2021

## Raquel Aoki

Starting the project


Data simulation sources: 
- https://github.com/JiajingZ/CopulaSensitivity
- https://github.com/raquelaoki/ParKCa

In [1]:
import sys 
sys.path.insert(0,'src/')
sys.path.insert(0,'bartpy/') #https://github.com/JakeColtman/bartpy
from data_simulation import *
from bartpy.sklearnmodel import SklearnModel as bart
from bartpy.features.featureimportance import feature_importance
from bartpy.features.featureselection import SelectNullDistributionThreshold, SelectSplitProportionThreshold
from bartpy.diagnostics.features import *

from sklearn.metrics import confusion_matrix,f1_score
from sklearn.metrics import roc_curve,roc_auc_score

def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value
    https://stackoverflow.com/questions/28719067/roc-curve-and-cut-off-point-python
    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 



/project/6007580/raoki/M3E2/env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
#sim_data = copula_simulated_data(s = 10)
#tr, u, y_continuous, y_binary = sim_data.get_data()
#effect_true, effect_obs, true_treat_obs_effect_01,true_treat_obs_effect= sim_data.get_true_coefs()
#sim_data.print_equation()


In [3]:
#change the proportion of true causes
gwas_data = gwas_simulated_data(1000, 100, 8, prop_tc = 0.05)
y, tc, X, col = gwas_data.generate_samples()
X = pd.DataFrame(X).sample(frac=1.0).values
y = y.astype('float')

In [4]:
model = bart(n_chains = 1, n_burn =200, n_trees=40, store_in_sample_predictions=False, n_jobs=1) # Use default parameters
model.fit(X, y) # Fit the model
#predictions = model.predict() # Make predictions on the train set
#out_of_sample_predictions = model.predict(X_test) # Make predictions on new data
#y_ = model.predict(X)
#thhold = Find_Optimal_Cutoff(y,y_)
#y_01 = [0 if item < thhold else 1 for item in y_]
#print('\n...Training set: F1 - ',f1_score(y,y_01))
#print('...... confusion matrix: ',confusion_matrix(y,y_01).ravel())
#...Training set: F1 -  0.5192743764172335
#...... confusion matrix:  [347 232 192 229]

  0%|          | 0/200 [00:00<?, ?it/s]

Starting burn


  2%|▏         | 3/200 [00:00<00:07, 27.55it/s]

Starting sampling


100%|██████████| 200/200 [00:06<00:00, 28.70it/s]


SklearnModel(n_chains=1, n_jobs=1, n_trees=40)

In [ ]:
#Feature_importance: 
#orginal, null = np.zeros(X.shape[1]),np.zeros(X.shape[1])
#for feature in range(X.shape[1]):
#    print(feature)
    #o, n = 
feature_importance(model, X, y,0)
    ##print(o,n)
    #original[feature], null[feature] = o.mean(), n.mean()

  1%|          | 2/200 [00:00<00:19, 10.39it/s]

Starting burn


  2%|▏         | 3/200 [00:00<00:06, 28.98it/s]

Starting sampling


  1%|          | 2/200 [00:00<00:19, 10.33it/s]

Starting burn


  2%|▏         | 3/200 [00:00<00:06, 28.74it/s]

Starting sampling


100%|██████████| 200/200 [00:06<00:00, 29.73it/s]


inside print [11.06918753973746, 11.266141699781482] 11.16766461975947


In [ ]:
def cate(y_pred, X, tc, col):
    cate_ = []
    for i in col: 
        y0 = np.array(y_)[X.iloc[:,i]==0]
        y1 = np.array(y_)[X.iloc[:,i]==1]
        cate_.append(y1.mean() - y0.mean())
        print(y1.mean() - y0.mean(), tc[i])
    pehe = sum(pow(tc[col]-cate_,2))/len(col)
    return cate_, pehe

cate_, pehe = cate(y_, X, tc, col)
#print(cate_)
print('PEHE',pehe)#0.1830

#for i in col: 
i = col[2]
y0 = y_[X.iloc[:,i]==0]
y1 = y_[X.iloc[:,i]==1]
print((y1.mean()- y0.mean())/len(y_), tc[i], i, y0.mean(),y1.mean())

In [ ]:
X1 = X.copy()
i = col[2]
X1.iloc[:,i] = 1-X1.iloc[:,i]
y_1 = model.predict(X1)
print(tc[i],y_1.mean()-y_.mean() )


In [ ]:
from bartpy.sklearnmodel import SklearnModel
from bartpy.features.featureselection import SelectNullDistributionThreshold, SelectSplitProportionThreshold
from bartpy.diagnostics.features import *
from bartpy.features.featureimportance import feature_importance


x = np.linspace(0, 5, 1000)
X = np.random.normal(0, 4, size = 1000 * 3).reshape(-1, 3)
X[:, 0] = x
X = pd.DataFrame(X).sample(frac=1.0).values
y = np.random.normal(0, 0.1, size=1000) + np.sin(X[:, 0])
model = SklearnModel(n_samples=200, n_burn=50, n_trees=10, store_in_sample_predictions=False, n_jobs=1)
model.fit(X, y)
original, null = feature_importance(model, X, y, 0)#
print(X.shape,y.shape)


/project/6007580/raoki/M3E2/env/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
 26%|██▌       | 13/50 [00:00<00:00, 127.62it/s]

Starting burn


  9%|▉         | 18/200 [00:00<00:01, 173.91it/s]

Starting sampling


 30%|███       | 15/50 [00:00<00:00, 147.79it/s]

...

 36%|███▌      | 18/50 [00:00<00:00, 172.74it/s]

Starting burn


  8%|▊         | 17/200 [00:00<00:01, 164.91it/s]

Starting sampling


 72%|███████▏  | 144/200 [00:00<00:00, 177.05it/s]

In [ ]:
a = np.random.normal(0, 0.1, size=1000) + np.sin(X[:, 0])


In [ ]:
y